Les acteurs ont en moyenne quel âge ? données films et acteurs (champs age, birth year et start year)
***age moyen à date du film

1- Appel des deux tables - trie ordre croissant - reset index et drop colonne en trop pour les deux tables

2- Nettoyage de la base Movies :
         - Conservation uniquement des films titleType = movie
         - Retrait des films isadult
         - Suppresion colonne quine me sont plus utiles
         - Explode genre en plusieurs colonnes

3- Nettoyage de la base Actors :
        - Supprimer les acteurs nés avant 1800 pour alléger la base
        - Explode colonne primaryProfession et conserver actor actress
        - Retirer la colonne deathYear
        - Explode colonne knownForTitles en film 1 film 2 etc....
        
        


3- Joindre les tables et retrier :
    - Supprimer les films plus anciens que 1918
    - Suprimer les lignes ou l'âge de l'acteur n'est pas renseigné
(ce tri n'a pas été fait avant car cela permet de nattoyer la base acteurs/movies en même temps)

4 - Créer une colonne âge acteur dans le film : Date du film - date de naissance (startYear - birthYear)

5 - il me faudra une moyenne de cette colonne pour répondre à la question par année






In [66]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns

## 1- Appel des trois tables - trie - reset index et drop colonne en trop pour les deux tables :
 actors = name.basics
 movies = title.basics
 title = 

## Table Actors

In [145]:
# appel de la table acteurs

actors = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = '\t', low_memory=False)

In [146]:
# tri des acteurs par date de naissance 
actors.sort_values(by=['birthYear'], inplace=True)

#reset index 
actors.reset_index(inplace=True)

#drop colonne index en double 
actors.drop('index',axis = 1, inplace=True)



In [147]:
# check si l'ordre est bien modifié et l'index modifié

actors.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0784172,Lucio Anneo Seneca,0004,0065,writer,"tt0049203,tt0972562,tt0218822,tt0237666"
1,nm4511421,Megan Liz Smith,0012,\N,"miscellaneous,actress","tt3416744,tt1966378,tt10183848"
2,nm8810146,Paul Walsh,0021,\N,actor,"tt0098878,tt0314567,tt0093223"
3,nm0430769,Flavius Josephus,0037,0095,"writer,miscellaneous","tt0273236,tt0237666,tt0009573,tt0049833"
4,nm1306202,Plutarch,0046,0122,writer,"tt0237666,tt0272979,tt0056937,tt0472473"


In [148]:
actors.shape # table brute

(11366782, 6)

## Table Movies

In [211]:
# appel de la table films (j'ai pris title basics qui contient l'année du film)

movies = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', low_memory=False)
movies.sort_values('startYear', inplace=True)

In [212]:
#reset index base movies
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [213]:
# check si l'ordre est bien modifié et l'index modifié
movies.head() 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt3155794,short,Passage de Venus,Passage de Venus,0,1874,\N,1,"Documentary,Short"
1,tt14495706,short,La Rosace Magique,La Rosace Magique,0,1877,\N,1,"Animation,Short"
2,tt2221420,short,Sallie Gardner at a Gallop,Sallie Gardner at a Gallop,0,1878,\N,1,"Documentary,Short"
3,tt12592084,short,Le singe musicien,Le singe musicien,0,1878,\N,1,"Animation,Short"
4,tt7816420,short,Athlete Swinging a Pick,Athlete Swinging a Pick,0,1881,\N,1,"Documentary,Short"


In [214]:
movies.shape # table brute

(8468615, 9)

## Table Title

In [75]:
# appel de la table title
title = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")

In [76]:
# filtre par category actor and actress
title = title.loc[(title['category']=='actor')|(title['category']=='actress')]

In [77]:
#reset index base title
title.reset_index(inplace=True)

#drop colonne index en double 
title.drop('index',axis = 1, inplace=True)

In [78]:
# remplacer les \n par des nan
title.replace('\\N', np.nan, inplace=True)

In [79]:
# check si l'index est bien modifié
title.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000005,1,nm0443482,actor,NaN,"[""Blacksmith""]"
1,tt0000005,2,nm0653042,actor,NaN,"[""Assistant""]"
2,tt0000007,1,nm0179163,actor,NaN,NaN
3,tt0000007,2,nm0183947,actor,NaN,NaN
4,tt0000008,1,nm0653028,actor,NaN,"[""Sneezing Man""]"


## 2-Nettoyage de la base Movies :
###          - Remplacer les \N par des nan
###          - Conservation uniquement des films titleType = movie
###          - Retrait des films isadult
###          - Suppresion des colonnes qui ne me sont plus utiles
###          - suppression des lignes aux données startYear et runtimeMinutes avec des nan
###          - change str en int sur les colonnes startYear et runtimeMinutes et tri sur durée + reset index
###          - Tri des valeurs par nconst + set index

In [215]:
# Remplacer les \N par des nan

movies.replace('\\N', np.nan, inplace=True)

In [216]:
#Conservation uniquement des films titleType = movie

movies = movies[movies['titleType'] == 'movie']

In [217]:
# Retrait des films isadult 

movies = movies[movies['isAdult'] == '0']

In [218]:
# supression des colonnes inutiles
movies = movies.drop(columns=['primaryTitle','titleType', 'endYear', 'isAdult'])

#movies = movies.drop(columns=['primaryTitle','originalTitle','titleType', 'endYear', 'isAdult', 'genres'])

In [219]:
movies.head(2)

,tconst,originalTitle,startYear,runtimeMinutes,genres
366,tt2210499,Birmingham,1896,61,Documentary
1753,tt0229676,Reproduction of the Corbett and Fitzsimmons Fight,1897,NaN,"Documentary,News,Sport"


In [220]:
# suppression des lignes aux données startYear et runtimeMinutes avec des nan

movies.dropna(subset=['startYear', 'runtimeMinutes' ], inplace=True)



In [221]:
# change str en int sur les colonnes startYear et runtimeMinutes
movies = movies.astype({'startYear': int})
movies = movies.astype({'runtimeMinutes': int})

In [222]:
#je ne conserve que les durées entre 58mn et inf à 270mn
movies = movies[(movies['runtimeMinutes']>=58) & (movies['runtimeMinutes']<=270)]

In [223]:
#reset index base movies (again !)
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [224]:
# trie par valeur nconst
movies = movies.sort_values('tconst')

#set_index = nconst
movies = movies.set_index('tconst')

In [225]:
#check
movies.head(2)

,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,
tt0000502,Bohemios,1905,100,NaN
tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"


In [226]:
# table uniquement avec les données conservées
movies.shape 

(326081, 4)

## 3- Nettoyage de la base Actors :
###        - Remplacer les \N par des nan
###        - Supprimer les lignes qui contiennent des nan dans la colonne birthYear  
###        - Retirer les colonnes qui ne me sont pas utiles
###        - Supprimer les acteurs nés avant 1800 + reset index
###        - change str en int sur les colonnes startYear et runtimeMinutes
###        - Tri des valeurs par nconst + set index

In [165]:
# remplacer les \N pas des nan 

actors.replace('\\N', np.nan, inplace=True)

In [166]:
# suppression des lignes qui comportent des Nan dans la colonne birthYear

actors.dropna(subset=['birthYear'], inplace=True)

In [167]:
# Retirer les colonnes qui ne me sont pas utiles
#actors.drop(columns=['deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [168]:
# ne conserve que les rows des acteurs/actrices nés avant 1800
actors = actors[actors['birthYear'] >= '1800']

#reset index base actors (again !)
actors.reset_index(inplace=True)

#drop colonne index en double (again !)
actors.drop('index',axis = 1, inplace=True)

In [169]:
# change str en int sur les colonnes startYear et runtimeMinutes
actors = actors.astype({'birthYear': int})

In [198]:
# Table actors - trie par valeur nconst
actors = actors.sort_values('nconst')

#set_index = nconst
actors = actors.set_index('nconst')

In [199]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536825 entries, nm0000001 to nm9993675
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   primaryName        536825 non-null  object
 1   birthYear          536825 non-null  int32 
 2   deathYear          183619 non-null  object
 3   primaryProfession  470261 non-null  object
 4   knownForTitles     523837 non-null  object
dtypes: int32(1), object(4)
memory usage: 22.5+ MB


In [200]:
# check index
actors.head()

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0071877,tt0117057"
nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0056404,tt0054452,tt0049189,tt0057345"
nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0078723,tt0077975,tt0072562"
nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0060827,tt0083922,tt0050976"


In [ ]:
# explode colonne primaryProfession
#actors[['firstProfession', 'secondProfession', 'thirdProfession']] = actors['primaryProfession'].str.split(',', n=2, expand=True)

#drop colonne primaryProfession
#actors.drop(columns=['primaryProfession'],axis = 1, inplace=True)

In [ ]:
# check dataframe actors
#actors.head()

In [ ]:
# Je ne garde que les acteurs/actrices de mes trois colonnes : firstProfession, secondProfession, thirdProfession

#actors = actors[(actors['firstProfession'].isin(['actress','actor'])) | ( actors['secondProfession'].isin(['actress','actor'])) | ( actors['thirdProfession'].isin(['actress','actor']))]

In [ ]:
#drop colums firstProfession, secondProfession, thirdProfession qui ne me sont plus utiles
#actors.drop(columns=['firstProfession','secondProfession', 'thirdProfession'],axis = 1, inplace=True)

## 3- Nettoyage de la base Title :
###        - Remplacer les \N par des nan
###        - dropna sur les cellules tconst non renseignées
###        - supprimer les colonnes inutiles
###        - Tri des valeurs par nconst + set index

In [172]:
# Remplacer les \N par des nan

title.replace('\\N', np.nan, inplace=True)

In [176]:
#dropna sur les cellules startyear non renseignées

title.dropna(subset=['tconst'],inplace=True)

In [178]:
title.head(2)

,tconst,nconst,category
0,tt0000005,nm0443482,actor
1,tt0000005,nm0653042,actor


In [ ]:
# suppression des colonnes inutiles

title.drop(columns=['ordering','job', 'characters'], inplace=True)

In [202]:
# trie par valeur nconst
title = title.sort_values('nconst')

#set_index = nconst
title = title.set_index('nconst')

In [203]:
title.head()

,tconst,category
nconst,,
nm0000001,tt0040308,actor
nm0000001,tt0612770,actor
nm0000001,tt0612774,actor
nm0000001,tt9708268,actor
nm0000001,tt0036363,actor


## 3- Joindre les tables et retrier :

In [204]:
# Table actors
actors.head(2)


,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0071877,tt0117057"


In [205]:
# table title:
title.head(2)

,tconst,category
nconst,,
nm0000001,tt0040308,actor
nm0000001,tt0612770,actor


In [208]:
# merge des tables actors et title 
df_actors_title = actors.merge(title , how='inner', on='nconst')

In [230]:
#check merge actors et title 
df_actors_title.head()

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,category
nconst,,,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0040308,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0612770,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0612774,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt9708268,actor
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0036363,actor


In [235]:
#reset index de la table merge

df_actors_title = df_actors_title.reset_index()

In [236]:
#check
df_actors_title.head(2)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,category
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0040308,actor
1,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",tt0612770,actor


In [237]:
#set index de la table merge avec la colonne tconst (pour mon prochain merge)
df_actors_title = df_actors_title.set_index('tconst')

In [238]:
#check
df_actors_title.head(2)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category
tconst,,,,,,,
tt0040308,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor
tt0612770,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor


In [229]:
# Table  movies
movies.head(2)

,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,
tt0000502,Bohemios,1905,100,NaN
tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"


In [239]:
# merge des tables df_actors_title  et movies

actors_title_movies = df_actors_title.merge(movies, how='inner', on='tconst' )


In [240]:
# check merge actors_title  et movies
actors_title_movies.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,,,,,,,,
tt0040308,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor,Easter Parade,1948,103,"Musical,Romance"
tt0040308,nm0000023,Judy Garland,1922,1969,"soundtrack,actress","tt0032138,tt0055031,tt0047522,tt0037059",actress,Easter Parade,1948,103,"Musical,Romance"
tt0040308,nm0492444,Peter Lawford,1923,1984,"actor,soundtrack,producer","tt0037988,tt0039431,tt0054135,tt0056197",actor,Easter Parade,1948,103,"Musical,Romance"
tt0040308,nm0587900,Ann Miller,1923,2004,"actress,soundtrack,miscellaneous","tt0045963,tt0041716,tt0036306,tt0166924",actress,Easter Parade,1948,103,"Musical,Romance"
tt0036363,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor,The Sky's the Limit,1943,89,"Comedy,Musical,Romance"


In [241]:
# calcul age acteur au moment du film + creation colonne 'age_actor_actress' pour le résultat

actors_title_movies['age_actor_actress'] = actors_title_movies['startYear'] - actors_title_movies['birthYear']


In [243]:
actors_title_movies.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category,originalTitle,startYear,runtimeMinutes,genres,age_actor_actress
tconst,,,,,,,,,,,,
tt0040308,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor,Easter Parade,1948,103,"Musical,Romance",49
tt0040308,nm0000023,Judy Garland,1922,1969,"soundtrack,actress","tt0032138,tt0055031,tt0047522,tt0037059",actress,Easter Parade,1948,103,"Musical,Romance",26
tt0040308,nm0492444,Peter Lawford,1923,1984,"actor,soundtrack,producer","tt0037988,tt0039431,tt0054135,tt0056197",actor,Easter Parade,1948,103,"Musical,Romance",25
tt0040308,nm0587900,Ann Miller,1923,2004,"actress,soundtrack,miscellaneous","tt0045963,tt0041716,tt0036306,tt0166924",actress,Easter Parade,1948,103,"Musical,Romance",25
tt0036363,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor,The Sky's the Limit,1943,89,"Comedy,Musical,Romance",44


## nettoyage df final : actors_title_movies

In [244]:
#reset index de la table pour que la Pk soit unique
actors_title_movies = actors_title_movies.reset_index()

In [249]:
#check
actors_title_movies.head()

,tconst,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,category,originalTitle,startYear,runtimeMinutes,genres,age_actor_actress
0,tt0040308,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor,Easter Parade,1948,103,"Musical,Romance",49
1,tt0040308,nm0000023,Judy Garland,1922,1969,"soundtrack,actress","tt0032138,tt0055031,tt0047522,tt0037059",actress,Easter Parade,1948,103,"Musical,Romance",26
2,tt0040308,nm0492444,Peter Lawford,1923,1984,"actor,soundtrack,producer","tt0037988,tt0039431,tt0054135,tt0056197",actor,Easter Parade,1948,103,"Musical,Romance",25
3,tt0040308,nm0587900,Ann Miller,1923,2004,"actress,soundtrack,miscellaneous","tt0045963,tt0041716,tt0036306,tt0166924",actress,Easter Parade,1948,103,"Musical,Romance",25
4,tt0036363,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983",actor,The Sky's the Limit,1943,89,"Comedy,Musical,Romance",44


In [251]:
#drop colonnes qui m'ont servi uniquement à vérifier mes merges
actors_title_movies.drop(columns=['tconst', 'nconst', 'primaryProfession','knownForTitles','runtimeMinutes', 'genres'], inplace=True)

In [252]:
#check
actors_title_movies.head()

,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
0,Fred Astaire,1899,1987,actor,Easter Parade,1948,49
1,Judy Garland,1922,1969,actress,Easter Parade,1948,26
2,Peter Lawford,1923,1984,actor,Easter Parade,1948,25
3,Ann Miller,1923,2004,actress,Easter Parade,1948,25
4,Fred Astaire,1899,1987,actor,The Sky's the Limit,1943,44


In [253]:
# je tri ici car cela permet de trier toutes les tables d'uns eul et éviter de faire 3 fois la même opé

actors_title_movies = actors_title_movies[(actors_title_movies['startYear'] >= 1918) & (actors_title_movies['startYear'] <= 2021)]

In [256]:
#check
actors_title_movies.shape

(658581, 7)

In [268]:
#film pour checker ensuite la moyenne

check_film = actors_title_movies[actors_title_movies['originalTitle'] == "Easter Parade"]


In [269]:
check_film

,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
0,Fred Astaire,1899,1987,actor,Easter Parade,1948,49
1,Judy Garland,1922,1969,actress,Easter Parade,1948,26
2,Peter Lawford,1923,1984,actor,Easter Parade,1948,25
3,Ann Miller,1923,2004,actress,Easter Parade,1948,25


### groupeby originalTitle sur moyenne des ages des acteurs (tous genres confondus)
### même chose pour les acteurs et actrices

In [262]:
# calcul moyenne age
age_mean = round(actors_title_movies.groupby('originalTitle').mean('age_actor_actress'))

In [279]:
# check df age_mean
age_mean.head()

,originalTitle,birthYear,startYear,age_actor_actress
0,#1 Serial Killer,1982.0,2013.0,31.0
1,"#73, Shaanthi Nivaasa",1962.0,2007.0,45.0
2,#BerhentiDiKamu,1952.0,2021.0,69.0
3,"#Bitch, les filles et la violence",1965.0,2015.0,50.0
4,#BlackSkin,1972.0,2021.0,49.0


In [273]:
#reset index
age_mean = age_mean.reset_index()

In [282]:
age_mean.drop(columns=['birthYear', 'startYear'], inplace=True)

In [283]:
#check df age_mean
age_mean.head()

,originalTitle,age_actor_actress
0,#1 Serial Killer,31.0
1,"#73, Shaanthi Nivaasa",45.0
2,#BerhentiDiKamu,69.0
3,"#Bitch, les filles et la violence",50.0
4,#BlackSkin,49.0


In [284]:
# selection du film : Easter Parade	
age_mean_film = age_mean[age_mean['originalTitle'] == "Easter Parade"]

In [285]:
#check moyenne age du film : Easter Parade	
age_mean_film

,originalTitle,age_actor_actress
45942,Easter Parade,31.0
